In [33]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [34]:
import pandas as pd

df = pd.read_csv('NSCH_2021E_TOPICAL.csv')

C:\Users\rm130166\AppData\Local\Temp\ipykernel_14356\2769899098.py:3: DtypeWarning: Columns (88,121,394,397,400,403,406,447) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('NSCH_2021E_TOPICAL.csv')


In [35]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50892 entries, 0 to 50891
Columns: 463 entries, FIPSST to WEIGHT
dtypes: float64(1), int64(43), object(419)
memory usage: 179.8+ MB
None


In [37]:
# Define categorical variables to be one-hot encoded
categorical_vars = ['SC_SEX', 'SC_RACE_R', 'TENURE', 'K4Q22_R', 'PHYSACTIV', 'OVERWEIGHT', 'K10Q40_R', 'K10Q11']

# Apply one-hot encoding
encoder = OneHotEncoder(drop='first', sparse=False)  # drop='first' to avoid multicollinearity
encoded_vars = pd.DataFrame(encoder.fit_transform(df[categorical_vars]), columns=encoder.get_feature_names_out(categorical_vars))

# Merge one-hot encoded variables with the main dataframe
df_encoded = pd.concat([df.drop(columns=categorical_vars), encoded_vars], axis=1)

# Inspect the new dataframe
print(df_encoded.head())


C:\Users\rm130166\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


   FIPSST STRATUM      HHID FORMTYPE  TOTKIDS_R HHLANGUAGE  SC_AGE_YEARS  \
0      48       1  21000002       T1          2          3             2   
1       2       1  21000009       T3          1          1            12   
2      40       1  21000017       T2          2          1             8   
3      26       1  21000030       T1          1          1             2   
4      22       1  21000031       T1          3          1             5   

  K2Q35A_1_YEARS BIRTH_MO BIRTH_YR  ... PHYSACTIV_M PHYSACTIV_N OVERWEIGHT_2  \
0              L        4     2020  ...         0.0         1.0          1.0   
1              L        6     2009  ...         0.0         0.0          1.0   
2              L        6     2013  ...         0.0         0.0          1.0   
3              L        1     2019  ...         0.0         1.0          1.0   
4              L        4     2016  ...         0.0         1.0          1.0   

  OVERWEIGHT_M K10Q40_R_2 K10Q40_R_3 K10Q40_R_4 K10Q40_R_M K10

In [38]:
# Define dependent variables (one-hot encoded of OVERWEIGHT, PHYSACTIV, K4Q22_R)
dependent_vars = encoded_vars.filter(regex='OVERWEIGHT_|PHYSACTIV_|K4Q22_R_')

# Define independent variables (one-hot encoded of K10Q11, K10Q40_R)
independent_vars = encoded_vars.filter(regex='K10Q11_|K10Q40_R_')

# Define covariates (one-hot encoded of SC_SEX, SC_RACE_R, TENURE and including SC_AGE_YEARS)
covariate_vars = encoded_vars.filter(regex='SC_SEX_|SC_RACE_R_|TENURE_')
covariate_vars['SC_AGE_YEARS'] = df['SC_AGE_YEARS']

# merge all these into a single dataframe
df_model = pd.concat([dependent_vars, independent_vars, covariate_vars], axis=1)

# Inspect the new dataframe
print(df_model.head())

   K4Q22_R_2  K4Q22_R_3  K4Q22_R_M  PHYSACTIV_2  PHYSACTIV_3  PHYSACTIV_4  \
0        0.0        1.0        0.0          0.0          0.0          0.0   
1        0.0        1.0        0.0          1.0          0.0          0.0   
2        0.0        1.0        0.0          1.0          0.0          0.0   
3        0.0        1.0        0.0          0.0          0.0          0.0   
4        0.0        1.0        0.0          0.0          0.0          0.0   

   PHYSACTIV_M  PHYSACTIV_N  OVERWEIGHT_2  OVERWEIGHT_M  ...  SC_SEX_2  \
0          0.0          1.0           1.0           0.0  ...       0.0   
1          0.0          0.0           1.0           0.0  ...       0.0   
2          0.0          0.0           1.0           0.0  ...       0.0   
3          0.0          1.0           1.0           0.0  ...       1.0   
4          0.0          1.0           1.0           0.0  ...       1.0   

   SC_RACE_R_2  SC_RACE_R_3  SC_RACE_R_4  SC_RACE_R_5  SC_RACE_R_7  TENURE_2  \
0          0

C:\Users\rm130166\AppData\Local\Temp\ipykernel_14356\4270881769.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covariate_vars['SC_AGE_YEARS'] = df['SC_AGE_YEARS']


In [39]:
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import pandas as pd

In [40]:
# Add a constant (intercept) to the covariates
covariate_vars = sm.add_constant(covariate_vars)

In [41]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Function to perform logistic regression, calculate odds ratios, and print the summary
def logistic_regression(dependent_var, independent_vars, covariates, outcome_name):
    # Combine independent variables and covariates
    X = pd.concat([independent_vars, covariates], axis=1)
    y = dependent_var.squeeze()  # Ensure y is a Series, not a DataFrame

    # Fit the logistic regression model
    model = sm.Logit(y, X)
    result = model.fit()
    
    # Calculate odds ratios and 95% confidence intervals
    odds_ratios = np.exp(result.params)
    conf = np.exp(result.conf_int())
    conf.columns = ['2.5%', '97.5%']
    
    # Print the model summary
    print(f"\nSummary for {outcome_name}:\n")
    print(result.summary())
    
    # Print odds ratios with confidence intervals
    print("\nOdds Ratios with 95% Confidence Intervals:\n")
    print(pd.concat([odds_ratios, conf], axis=1))

# Perform logistic regression for each dependent variable's column
for outcome in ['OVERWEIGHT_', 'PHYSACTIV_', 'K4Q22_R_']:
    dependent_var_columns = df_model.filter(regex=f'^{outcome}')
    
    # Iterate over each one-hot encoded column within the outcome
    for col in dependent_var_columns.columns:
        dependent_var = dependent_var_columns[[col]]
        logistic_regression(dependent_var, independent_vars, covariate_vars, col)


Optimization terminated successfully.
         Current function value: 0.221256
         Iterations 7

Summary for OVERWEIGHT_2:

                           Logit Regression Results                           
Dep. Variable:           OVERWEIGHT_2   No. Observations:                50892
Model:                          Logit   Df Residuals:                    50875
Method:                           MLE   Df Model:                           16
Date:                Tue, 24 Sep 2024   Pseudo R-squ.:                 0.07195
Time:                        21:37:21   Log-Likelihood:                -11260.
converged:                       True   LL-Null:                       -12133.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
K10Q40_R_2      -0.3207      0.042     -7.669      0.000      -0.403      -0

Optimization terminated successfully.
         Current function value: 0.405157
         Iterations 6

Summary for PHYSACTIV_3:

                           Logit Regression Results                           
Dep. Variable:            PHYSACTIV_3   No. Observations:                50892
Model:                          Logit   Df Residuals:                    50875
Method:                           MLE   Df Model:                           16
Date:                Tue, 24 Sep 2024   Pseudo R-squ.:                  0.1076
Time:                        21:37:22   Log-Likelihood:                -20619.
converged:                       True   LL-Null:                       -23106.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
K10Q40_R_2      -0.1025      0.030     -3.445      0.001      -0.161      -0.

C:\Users\rm130166\AppData\Local\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\rm130166\AppData\Local\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\rm130166\AppData\Local\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\rm130166\AppData\Local\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)


         Current function value: 0.000000
         Iterations: 35

Summary for PHYSACTIV_N:



C:\Users\rm130166\AppData\Local\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


                           Logit Regression Results                           
Dep. Variable:            PHYSACTIV_N   No. Observations:                50892
Model:                          Logit   Df Residuals:                    50875
Method:                           MLE   Df Model:                           16
Date:                Tue, 24 Sep 2024   Pseudo R-squ.:                   1.000
Time:                        21:37:24   Log-Likelihood:            -5.4594e-08
converged:                      False   LL-Null:                       -34392.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
K10Q40_R_2      -3.2128   4.66e+04   -6.9e-05      1.000   -9.13e+04    9.12e+04
K10Q40_R_3      -3.3035   1.08e+05  -3.07e-05      1.000   -2.11e+05    2.11e+05
K10Q40_R_4     -10.4770   7.31e+04     -0.00


Summary for K4Q22_R_M:

                           Logit Regression Results                           
Dep. Variable:              K4Q22_R_M   No. Observations:                50892
Model:                          Logit   Df Residuals:                    50875
Method:                           MLE   Df Model:                           16
Date:                Tue, 24 Sep 2024   Pseudo R-squ.:                 0.07393
Time:                        21:37:25   Log-Likelihood:                -1491.1
converged:                       True   LL-Null:                       -1610.2
Covariance Type:            nonrobust   LLR p-value:                 1.439e-41
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
K10Q40_R_2       0.1716      0.160      1.069      0.285      -0.143       0.486
K10Q40_R_3      -0.0874      0.459     -0.190      0.849      -0.988       0.813
K10Q40_R_4       0.